<a href="https://colab.research.google.com/github/Harshkotkar/Deep-Learning/blob/main/advance_rag(webbase_search).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip show langchain

Name: langchain
Version: 1.0.2
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 


In [30]:
!pip install arxiv
!pip install wikipedia
!pip install langchain-google-genai


In [31]:
!pip install langchain_community
!pip install langchain-text-splitter
!pip install faiss-cpu


ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter


In [32]:
import getpass
import os
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")


Enter your Google API key: ··········


In [68]:
import os
import getpass

# 🔹 Enable LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# 🔹 Ask user for their LangSmith API key (securely)
langsmith_key = getpass.getpass("🔑 Enter your LangSmith API key: ")
os.environ["LANGCHAIN_API_KEY"] = langsmith_key

print("✅ Environment variables set for LangSmith ")

🔑 Enter your LangSmith API key: ··········
✅ Environment variables set for LangSmith 


In [33]:
from langchain_community.tools import WikipediaQueryRun# langchain_community consiist all the tools are available in langchain
from langchain_community.utilities import WikipediaAPIWrapper

In [34]:
wiki_api=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)#basically it is interacting with wiki to find the multipule results and then it is returning the top k results based on our query
wiki=WikipediaQueryRun(api_wrapper=wiki_api)# it is taking the wiki api wrapper as input and then it is returning the tool which we can use to query the wikipedia

In [35]:
wiki.name

'wikipedia'

In [36]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS#use to create the vector storage
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter #split text into chunks
embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5",
                                 model_kwargs={"device":"cpu"},
                                 encode_kwargs={"normalize_embeddings":True})
#create the text into numberical embeddings with semantic meaning in 768 dimension vector

loder=WebBaseLoader("https://docs.langchain.com/langsmith/home")#it is used to load the data from the web page
docs=loder.load()#it is used to load the data from the web page

documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs) #it is used to split the text into chunks of 1000 characters with an overlap of 200 characters overlap means that last 200 characters of previous chunk will be same as first 200 characters of next chunk
vectordb=FAISS.from_documents(documents,embedding=embeddings)#it is used to create the vector storage from the documents and embeddings
retriver=vectordb.as_retriever()#it is a interface which is used to retrieve the documents from the vector storage based on the query
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e17d6535f70>, search_kwargs={})

In [37]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# loder=WebBaseLoader("https://docs.langchain.com/langsmith/home")#it is used to load the data from the web page
# docs=loder.load()#it is used to load the data from the web page

# documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs) #it is used to split the text into chunks of 1000 characters with an overlap of 200 characters overlap means that last 200 characters of previous chunk will be same as first 200 characters of next chunk
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# vectordb=FAISS.from_documents(documents,embedding=embeddings)#it is used to create the vector storage from the documents and embeddings
# retriver=vectordb.as_retriever()#it is a interface which is used to retrieve the documents from the vector storage based on the query
# retriver


In [38]:
#create retriver tool use to do retieval of the documents
from langchain_core.tools import create_retriever_tool
retriever_tool=create_retriever_tool(retriever=retriver,name="langsmith_search",
                      description='''Search for information about the langsmith. for any question about langsmith, you must use this tool''')

In [39]:
retriever_tool.name

'langsmith_search'

In [40]:
# Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)#it is used to interact with arxiv api to get the research papers based on our query, doc_content_chars_max is used to limit the number of characters in the document
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)#it is used to create the tool which we can use to query the arxiv api
arxiv_tool.name

'arxiv'

In [41]:
tools=[wiki,arxiv_tool,retriever_tool]

In [42]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.12/dist-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=300)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about the langsmith. for any question about langsmith, you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7e17575e6700>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.

In [43]:
### agents will be responsible to use these tools to answer the questions basically in agents language model is used to decied which tool to use and when to use (like it use the reasonong engine to determine which action to take in which order) where the chains are used to perfrom specific tasks chains are hardcoded to a specific sequence of actions.

# basicaly here in tools its like it will seach first in wiki if it dont find anything then it will search in arxiv if it dont find anything then it will search in langsmith documents retriver tool.😁

In [44]:
from langchain_google_genai import ChatGoogleGenerativeAI


google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    os.environ["GOOGLE_API_KEY"] = google_api_key

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0)


In [45]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate
)

# System message: defines the assistant's role
system_message = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=[],
        template="You are a helpful assistant that answers questions about LangSmith."
    )
)

# Placeholder for chat history (optional)
chat_history = MessagesPlaceholder(variable_name="chat_history", optional=True)

# Human message: takes user input
human_message = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["input"],
        template="{input}"
    )
)

# Scratchpad for intermediate agent steps
scratchpad = MessagesPlaceholder(variable_name="agent_scratchpad")

# Combine into a full chat prompt
prompt = ChatPromptTemplate.from_messages([
    system_message,
    chat_history,
    human_message,
    scratchpad
])


In [93]:
# from langchain.agents import create_tool_calling_agent, AgentExecutor
# agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
# agent_executor = AgentExecutor(agent=agent, tools=tools)

# # Run the agent
# response = agent_executor.invoke({"input": "How do I trace a LangChain run with LangSmith?"})
# print(response)


# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain.tools import tool
# from langchain.agents import create_agent
# from langgraph.checkpoint.memory import MemorySaver

# memory=MemorySaver()
# agent=create_agent(model=llm,tools=tools,checkpointer=memory)
# response = agent.invoke({
#     "messages": [("human", "How do I trace a LangChain run with LangSmith?")]
# })

# final msg
# print(response["messages"][-1].content)

In [94]:
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain.tools import tool
# from langgraph.prebuilt import create_react_agent
# from langgraph.checkpoint.memory import MemorySaver




# agent = create_react_agent(model=llm, tools=tools,prompt=prompt)

# response = agent.invoke({
#     "messages": [("human", "what is LangSmith?")]
# })

# print(response["messages"][-1].content)

# # from langchain.agents import AgentExecutor
# # agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
# # agent_executor
# # agent_executor.invoke({"input":"tell me about langsmith"})


In [95]:
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langgraph.prebuilt import create_react_agent
# from langgraph.checkpoint.memory import MemorySaver

# tools = [wiki, arxiv_tool, retriever_tool]

# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0)

# memory = MemorySaver()

# agent = create_react_agent(
#     model=llm,
#     tools=tools,
#     prompt=prompt,
#     checkpointer=memory  # memory = conversation state
# )

# config = {"configurable": {"thread_id": "harsh_langsmith_session_1"}}

# response = agent.invoke(
#     {"input": "What is LangSmith?", "agent_scratchpad": ""},
#     config=config
# )

# print( response["messages"][-1].content)

# response2 = agent.invoke(
#     {"input": "Who created it?", "agent_scratchpad": ""},
#     config=config
# )

# print( response2["messages"][-1].content)


In [89]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# Memory
memory = MemorySaver()

agent = create_react_agent(
    model=llm,
    tools=tools,
    checkpointer=memory
)

#
config = {"configurable": {"thread_id": "session_1"}}



#final response
response = agent.invoke(
    {"messages": [("human", "What is paper 1605.08386 about?")]},
    config=config
)

print("\n\n Final answer:", response)


/tmp/ipython-input-1048840565.py:17: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(




 Final answer: {'messages': [HumanMessage(content='What is paper 1605.08386?', additional_kwargs={}, response_metadata={}, id='0f15f1bc-dd5b-4042-861c-fae8b19a7b7e'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'arxiv', 'arguments': '{"query": "1605.08386"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-exp', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--ee3142ef-6f5e-454c-bf4f-0458964e796a-0', tool_calls=[{'name': 'arxiv', 'args': {'query': '1605.08386'}, 'id': '8867d702-993a-4372-b373-3cc9503ddbc8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 157, 'output_tokens': 12, 'total_tokens': 169, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Published: 2016-05-26\nTitle: Heat-bath random walks with Markov bases\nAuthors: Caprice Stanley, Tobias Windisch\nSummary: Graphs on lattice points are stud

In [92]:

import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# Tools
tools = [wiki, arxiv_tool, retriever_tool]

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0)

memory = MemorySaver()

#create Agent
agent = create_react_agent(
    model=llm,
    tools=tools,
    checkpointer=memory
)

# config (for conversation thread)
config = {"configurable": {"thread_id": "harsh_langsmith_session_1"}}

# run the query
response = agent.invoke(
    {"messages": [("human", "What is paper 1605.08386 about?")]},
    config=config
)

# output format

print("AGENT EXECUTION TRACE")


for msg in response["messages"]:
    if "function_call" in msg.additional_kwargs:
        fn = msg.additional_kwargs["function_call"]["name"]
        args = msg.additional_kwargs["function_call"]["arguments"]
        print(f"\n Function Call → {fn}({args})")

    # check if it's a tool's output
    elif msg.__class__.__name__ == "ToolMessage":
        print("\n Tool Output:")
        print(msg.content.strip())

# final ans
final_answer = response["messages"][-1].content
print("\n Final Answer:")
print(final_answer)
print("=" * 80)


/tmp/ipython-input-1597741302.py:17: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


AGENT EXECUTION TRACE

 Function Call → arxiv({"query": "1605.08386"})

 Tool Output:
Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allo

 Final Answer:
The paper 1605.08386, titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch, is about graphs on lattice points whose edges come from a finite set.
